<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Nama: Derfansyah Guswiranata Kurnia
* Identitas Dicoding : dgkurnia
* email dicoding : weerakurnia@gmail.com
* Alamat: Jalan Malaka Hijau No 39 Perumahan Malaka Country Estate Jakarta Timur

#Pembaruan

In [76]:
#perkembangan google colab
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 3s (92.2 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [77]:
#request dan pandas
!pip install requests pandas

In [78]:
#instalasi sastarawi
!pip install sastrawi

In [79]:
#instalasi wordcloud
!pip install wordcloud

In [80]:
#instalasi backoff
!pip install backoff aiohttp

# Impor Perpustakaan

In [81]:
#untuk akses ke google drive (bila diperlukan)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
#instalasi perpustakaan
import requests
import json
#Bagian Pandas
import pandas as pd
pd.options.mode.chained_assignment = None #pengaturan peringatan 'rantai'
import csv
import string
#ke manajemen

In [83]:
#manajemen data dasar
import numpy as np
seed = 0 #reprodubilitas
np.random.seed(seed)
#ke plt

In [84]:
#Bagian plt
import matplotlib.pyplot as plt
import datetime as dt #jika ada tanggal
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d
#waktu
import time
#ke sastawi, NLTK, dan wordcloud

In [85]:
#Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #bagian 'stemmer factory'
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [86]:
#sastrawi dan NITK [NITK]
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.
from nltk.corpus import stopwords  # Mengimpor modul stopwords dari pustaka NLTK.
from nltk.tokenize import word_tokenize  # Mengimpor fungsi word_tokenize dari modul tokenize dalam pustaka NLTK.

#impor wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
#ke sastrawi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Impor Teknik

In [87]:
#pemisahan data dengan 'train test split'
from sklearn.model_selection import train_test_split

In [88]:
#Teknik untuk inspeksi: perpustakaan untuk klasifikasi
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

In [89]:
#Bagian re IO bentuk string
import re
from io import StringIO

In [90]:
#metrik penilaian (akurasi, lenyap/loss, nilai f1, nilai recall)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Persiapan Ekstraksi situs

##Pendaftaran situs

In [91]:
#deklarasi 'URL target' dan asal URL
turldoma = 'https://gql.tokopedia.com/graphql/productReviewList'
asal = 'https://www.tokopedia.com' #semua dari Tokopedia
#ke pendaftaran URL

In [92]:
"""
#Daftar 'URL referensi' dan 'cookie' di Tokopedia
#daftar URL referensi
"""
urllist = ["https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-nessie-dark-green-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gruney-katun-red-m/review", "https://www.tokopedia.com/erigo/kaos-erigo-t-shirt-oversize-antelope-cotton-combed-black-m/review"]
#ke cookie

In [93]:
#pendaftaran cookie (dari atas ke bawah: a,b,c,d,e,f)
"""
cooklist = ["_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; _hjSession_714968=eyJpZCI6ImVkMDdhZTczLTgzZmQtNDJiZS1iN2FlLTU0ZWEzMGVkMWRmMiIsImMiOjE3Mzc2NzMxOTU0OTUsInMiOjEsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxfQ==; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyvu2AJWUAQAAzRp4lRpsSK9Kq9IKEENQmF/NP0n81a4G9NW2z1k3l0EdQf8ZoypFgm3B1ftEaI4VqiM6tdlXanArsn4PVjfxjVZ37K08ajCP5hsWmqZnfgkbuPo27GCggtcgw7GC7PFKThXlJkMSawZfzEkjtIFYDoyLOXUoqnbDVarjt7IK99YGIXqrDFPs9DNyn7cqCegduQbkpKQth4SFdyMOJ17gugVxzx1GSsx+QcR1Qtfa9Mcf2mme~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737674925795.17%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737674925.57.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=231d1a0a-4caf-4293-8ee5-445a599b6786",
            "_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675011304.19%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675011.55.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=b6a6fad4-1d28-40d4-a059-e96aeb37c794",
            "_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675112541.21%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675112.19.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=db5340ce-e46c-4570-8ff2-eea15fef8204",
            "_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675229289.23%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675229.48.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=2afacd32-857a-4612-8138-0f90f5fe16cc",
            " _gcl_au=1.1.1639389662.1737713243; _UUID_NONLOGIN_=121bb529a74834442acf6c81b5e09f6c; hfv_banner=true; _gid=GA1.2.1683694316.1737713243; _SID_Tokopedia_=DaTj-m583Yd83uszVV1bpFMY5f6kxR6OtaXytjTmvr48DJKl-hmDxy2KuwFe8kRaiHQ_Gwu5BAxSJqUUomOSlErgNxDiT6ZCByLuwn_ti43GZbXJ5vDVVxcHBbuTI_ne; DID=4b056bb420287f9ec4ae4848579bbb0551e0e7338740a5ac0ea671abdc8f10f1d4c55c9d04876bf416c20a99a5f6e2f3; DID_JS=NGIwNTZiYjQyMDI4N2Y5ZWM0YWU0ODQ4NTc5YmJiMDU1MWUwZTczMzg3NDBhNWFjMGVhNjcxYWJkYzhmMTBmMWQ0YzU1YzlkMDQ4NzZiZjQxNmMyMGE5OWE1ZjZlMmYz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; ak_bmsc=82C3E4FBC1F435D1042B5CFEF139563D~000000000000000000000000000000~YAAQ5y2pylKuXGqUAQAAZdrHlxoGYEc1MUW4hh9VCnd4/arQZBSX2AXz44UZcPTCijUcqVwKiIx3s0nIOaTxYunDhZVAomDzZEvr5bnY44jpvpHWYe9gMki93xFX5n+/LNmW8jQQzgTw7346xT65MwLnQ0GTlynrOiKulxEzkRyt9TX/KP84am26ZadVUcg8poXdvDPNjX2Q4BDpWeMboxZv/FNSSVmPPMx+bDoqMth6KqVgxn/pfE419hyoP6DlRLsrhwXtamNhPLDa+w1bbnzR5MVJYlb6uZE08xwfXsE2f8qMwEuAXg0cSuxv24Dp1KZrz4O+sUbHc1UzjdgVNafCMatzcEFZVgiBxdNOFaZ98HoK8c8L7jjC8ZJ4gnM39SvIRB9JWiO0NFfFtBFV5eLZCpdCypZzrbBE3ALgMcIKngxDp3l4LNyKAwd5uY7WE575+4sDTcw4k9pwRQ8L; _tt_enable_cookie=1; _ttp=l5D7lbwu4aAQYkIQZTVFCegxzBc.tt.1; AMP_TOKEN=%24NOT_FOUND; _fbp=fb.1.1737713248500.158953258786947229; TOPATK=7zc_0Hg8TgOW1zcFE2KV8w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; FPF=1; uide=pI7BGrsyMVJyePibts4sZZa6FDBGuydJEUT0Xdf+hc4/be7+; _abck=A3E412D90A6099E29D306E267A5C41E0~0~YAAQ5y2pyqGyXGqUAQAA+bzIlw30vKqekH2BBb9bkfFNBMv3w9g14QJp/EY1ix8P9JyJZmk7z2ieQT6wJP1kcZl/9a5z884LsAElieZ//2yOXKqgHe1++BOj/ABIAEkx2QCAy9uqyGtDC18L5UGcIlKxHtd8qva2/qmPtFsrhw1JVMkvnONx1Le4tYiS1/tfAjPIURknAlrI7t9sAfjQTAVED6a4vVsb2N65wjf457gORUbfl6NBWW8WaBoei2Dupobj6rnw5iiIHXia+IU7KJ1+M5I7adRU7IgB99PHvsI6tVtW9v8r4G4jiJ3UGxofxNkUryWuMozM6BXjDQtWcZh0yFsE12uteq5QB3GcwVgTzM1Pz/ibWlPON+AKfPu5aHvs4VH+3kx+coK9vNLTlDxdNYJ0YWHz/hifA3ky2lU9jn4Hj9HS3+D45GLz87gPz0+YJlEdBFNvBlrG4aWeG4Eq+AN/bDOseoAbmGte4S9ryw8v9KenrjhSD5nwgeofo3KGEe5jMG/ALdUl0T2xMjlbmFoCjUJh1WQs/sMHwwBeCqsr2C5NHBWGjFyn9QLTA6CDA4r6sz4OvAEXXi/KLfImDUEhaGDanZDtbpBfbKdHVgPNTjPztBhiue0FOckiedrMXqDXlRP2F4+b7jH5/0ORH+qhtvcSABBxa0tgwoTLymbfwRP+1NSGzuJJ1EkReHqig3KzKeBa6R7OQhM=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e313757323439333b393130283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=f0ab12ec-96bb-4c29-a405-7d22c65f4661; bm_sv=FD00D167DBF2190289DB0BE0959F6824~YAAQ1i2pyh+7CpWUAQAA/b/QlxpzX1HYcRWDsW54jwa08Z1CZQgoRy5iDvkrXamhqE4zmLRrf4flzhna5R1Cd1uIf52DTJm3rnI1SuPIgw+yVezataLTsFegYr0cf5tSVT/ESqzc463iT1JXH7r+6HjiRaQmODg+a7q+hjqK14R7yuYBUVsIXsR5XEbaLqn5+y7HDCTP1381QJzQyk1+CCa8oQnnxzsCbLYZXNAb88BQA5tCFcpq3qqrLhMN7NZe4mbg~1; bm_sz=D1E57C24BFE5447079B254F515DBD59A~YAAQD+nOFxdD95CUAQAAV37llxqatZcudbg3kOkx+Ux7+E/ljRrghY+y3avONARz753pJWXlHasXWewJDkw94OEbjrkBhBcjr7tIHaONBOI844nIGKH8oEg8jadNA5Ylt4xZVqEo+0y8feyRSUYUZSogQrFXBbpAnfAhQuM+YJdsFPfkHxYJQwtRsMDE20CSFKvyCTCmwTtUfIqAHQWMMfJljJVdy+dY8eH/QJSQcPtkPEY0vx5P/VpPAwSszojewxd/JwHi46MEOKj1c46I3vtM/NK7V2ujliS4YGGYpu3sN7u8qWikJV4b/nM+Z63NzGiCCXoZyoIFFA3Q/RsKPi6jecylapyLqt7D67gFqdUYlGOu4sanox2Qg8sin2opu5h5EFlNCdtMG8yVtL/wiaL6CNyFOj/DR83qCZcZymBJ8QIkCcTwgnLDix/ipm3SRwBEeRXijhdltyPaBSpXiMQrgWjpz5//Az7ARN7t77ANlrrc3G4x4QTAKDrfOg==~3227970~3159095; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.8109202435e7e49eaa74f4f7a9fca585.1737713243405.1737713243405.1737715347261.17%22%7D; _ga_70947XW48P=GS1.1.1737713243.1.1.1737715347.6.0.0; _ga=GA1.1.1612239045.1737713243; webauthn-session=b3774ea3-7d42-4271-bfda-5567909804a2",
            " _gcl_au=1.1.1639389662.1737713243; _UUID_NONLOGIN_=121bb529a74834442acf6c81b5e09f6c; hfv_banner=true; _gid=GA1.2.1683694316.1737713243; _SID_Tokopedia_=DaTj-m583Yd83uszVV1bpFMY5f6kxR6OtaXytjTmvr48DJKl-hmDxy2KuwFe8kRaiHQ_Gwu5BAxSJqUUomOSlErgNxDiT6ZCByLuwn_ti43GZbXJ5vDVVxcHBbuTI_ne; DID=4b056bb420287f9ec4ae4848579bbb0551e0e7338740a5ac0ea671abdc8f10f1d4c55c9d04876bf416c20a99a5f6e2f3; DID_JS=NGIwNTZiYjQyMDI4N2Y5ZWM0YWU0ODQ4NTc5YmJiMDU1MWUwZTczMzg3NDBhNWFjMGVhNjcxYWJkYzhmMTBmMWQ0YzU1YzlkMDQ4NzZiZjQxNmMyMGE5OWE1ZjZlMmYz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; ak_bmsc=82C3E4FBC1F435D1042B5CFEF139563D~000000000000000000000000000000~YAAQ5y2pylKuXGqUAQAAZdrHlxoGYEc1MUW4hh9VCnd4/arQZBSX2AXz44UZcPTCijUcqVwKiIx3s0nIOaTxYunDhZVAomDzZEvr5bnY44jpvpHWYe9gMki93xFX5n+/LNmW8jQQzgTw7346xT65MwLnQ0GTlynrOiKulxEzkRyt9TX/KP84am26ZadVUcg8poXdvDPNjX2Q4BDpWeMboxZv/FNSSVmPPMx+bDoqMth6KqVgxn/pfE419hyoP6DlRLsrhwXtamNhPLDa+w1bbnzR5MVJYlb6uZE08xwfXsE2f8qMwEuAXg0cSuxv24Dp1KZrz4O+sUbHc1UzjdgVNafCMatzcEFZVgiBxdNOFaZ98HoK8c8L7jjC8ZJ4gnM39SvIRB9JWiO0NFfFtBFV5eLZCpdCypZzrbBE3ALgMcIKngxDp3l4LNyKAwd5uY7WE575+4sDTcw4k9pwRQ8L; _tt_enable_cookie=1; _ttp=l5D7lbwu4aAQYkIQZTVFCegxzBc.tt.1; AMP_TOKEN=%24NOT_FOUND; _fbp=fb.1.1737713248500.158953258786947229; TOPATK=7zc_0Hg8TgOW1zcFE2KV8w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; FPF=1; uide=pI7BGrsyMVJyePibts4sZZa6FDBGuydJEUT0Xdf+hc4/be7+; _abck=A3E412D90A6099E29D306E267A5C41E0~0~YAAQ5y2pyqGyXGqUAQAA+bzIlw30vKqekH2BBb9bkfFNBMv3w9g14QJp/EY1ix8P9JyJZmk7z2ieQT6wJP1kcZl/9a5z884LsAElieZ//2yOXKqgHe1++BOj/ABIAEkx2QCAy9uqyGtDC18L5UGcIlKxHtd8qva2/qmPtFsrhw1JVMkvnONx1Le4tYiS1/tfAjPIURknAlrI7t9sAfjQTAVED6a4vVsb2N65wjf457gORUbfl6NBWW8WaBoei2Dupobj6rnw5iiIHXia+IU7KJ1+M5I7adRU7IgB99PHvsI6tVtW9v8r4G4jiJ3UGxofxNkUryWuMozM6BXjDQtWcZh0yFsE12uteq5QB3GcwVgTzM1Pz/ibWlPON+AKfPu5aHvs4VH+3kx+coK9vNLTlDxdNYJ0YWHz/hifA3ky2lU9jn4Hj9HS3+D45GLz87gPz0+YJlEdBFNvBlrG4aWeG4Eq+AN/bDOseoAbmGte4S9ryw8v9KenrjhSD5nwgeofo3KGEe5jMG/ALdUl0T2xMjlbmFoCjUJh1WQs/sMHwwBeCqsr2C5NHBWGjFyn9QLTA6CDA4r6sz4OvAEXXi/KLfImDUEhaGDanZDtbpBfbKdHVgPNTjPztBhiue0FOckiedrMXqDXlRP2F4+b7jH5/0ORH+qhtvcSABBxa0tgwoTLymbfwRP+1NSGzuJJ1EkReHqig3KzKeBa6R7OQhM=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e313757323439333b393130283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=f0ab12ec-96bb-4c29-a405-7d22c65f4661; bm_sv=FD00D167DBF2190289DB0BE0959F6824~YAAQ1i2pyh+7CpWUAQAA/b/QlxpzX1HYcRWDsW54jwa08Z1CZQgoRy5iDvkrXamhqE4zmLRrf4flzhna5R1Cd1uIf52DTJm3rnI1SuPIgw+yVezataLTsFegYr0cf5tSVT/ESqzc463iT1JXH7r+6HjiRaQmODg+a7q+hjqK14R7yuYBUVsIXsR5XEbaLqn5+y7HDCTP1381QJzQyk1+CCa8oQnnxzsCbLYZXNAb88BQA5tCFcpq3qqrLhMN7NZe4mbg~1; bm_sz=D1E57C24BFE5447079B254F515DBD59A~YAAQD+nOFxdD95CUAQAAV37llxqatZcudbg3kOkx+Ux7+E/ljRrghY+y3avONARz753pJWXlHasXWewJDkw94OEbjrkBhBcjr7tIHaONBOI844nIGKH8oEg8jadNA5Ylt4xZVqEo+0y8feyRSUYUZSogQrFXBbpAnfAhQuM+YJdsFPfkHxYJQwtRsMDE20CSFKvyCTCmwTtUfIqAHQWMMfJljJVdy+dY8eH/QJSQcPtkPEY0vx5P/VpPAwSszojewxd/JwHi46MEOKj1c46I3vtM/NK7V2ujliS4YGGYpu3sN7u8qWikJV4b/nM+Z63NzGiCCXoZyoIFFA3Q/RsKPi6jecylapyLqt7D67gFqdUYlGOu4sanox2Qg8sin2opu5h5EFlNCdtMG8yVtL/wiaL6CNyFOj/DR83qCZcZymBJ8QIkCcTwgnLDix/ipm3SRwBEeRXijhdltyPaBSpXiMQrgWjpz5//Az7ARN7t77ANlrrc3G4x4QTAKDrfOg==~3227970~3159095; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.8109202435e7e49eaa74f4f7a9fca585.1737713243405.1737713243405.1737715293578.16%22%7D; _dc_gtm_UA-9801603-1=1; _ga_70947XW48P=GS1.1.1737713243.1.1.1737715293.60.0.0; _ga=GA1.1.1612239045.1737713243; webauthn-session=02678496-f1f1-45e8-8bdc-7ea6e24fd215"]
"""

'\ncooklist = ["_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIH

In [94]:
#persiapan 'identitas produk'
idlist =  ["7576252307", "1624334615", "7576433902",  "1624293521",  "1624323568",  "2390937942" ] #enam sampel ini dari Erigo
#urutan dari kiri ke kanan : [a,b,c,d,e,f]
#ke fragmentasi

In [95]:
#fragmentasi data untuk kemudahan
[urla, urlb, urlc, urld, urle, urlf] = urllist #daftar URL referensi
#[cookiea, cookieb, cookiec, cookied, cookiee, cookief] = cooklist #daftar cookie
[ida, idb, idc, idd, ide, idf] = idlist #daftar ID produk
#ke persiapan target dan kepala

# Proses Iterasi untuk data

## Persiapan

### Pembentukan kepala dan **'bentuk kueri'**

In [96]:
#Persiapan kepala dan kueri yang distandardisasi
def stdheader(url, idpr): #idpr adalah identitas produk
  halaman_awal = 1 #di kueri
  kplstd = {'accept': '*/*',
   'accept-language': 'en-US,en;q=0.9',
   'bd-device-id': '7463084441910052353',
   'content-type': 'application/json',
   'dnt': '1' ,
   'origin': asal,
   'priority': 'u=1, i',
   'referer': url,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
   'sec-ch-ua-mobile': '?0',
   'sec-ch-ua-platform': '"Windows"',
   'sec-fetch-dest': 'empty',
   'sec-fetch-mode': 'cors',
   'sec-fetch-site': 'same-site',
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
   'x-price-center': 'true',
   'x-source': 'tokopedia-lite',
   'x-tkpd-lite-service': 'zeus',
   'x-version': '1c2c0dd'}
  return kplstd

In [97]:
#variasi kepala untuk iterasi [dari indeks 0 ke lima dengan arah kiri ke kanan]
kplstdlist = [stdheader(urla, ida), stdheader(urlb, idb), stdheader(urlc, idc), stdheader(urld, idd), stdheader(urle, ide), stdheader(urlf, idf)]
#ke persiapan bentuk

### Iterasi utama

In [98]:
# Proses iterasi produk
def datagenerator(url, idpr):
    # Pemanggilan fungsi pertama (stdheader)
    header = stdheader(url, idpr)
    # Persiapan bentuk untuk iterasi
    daftar_teks = []
    daftar_nilai = []
    daftar_label = []

    # Batas halaman untuk ulasan
    awal = 1  # dari halaman 1
    akhir = 50  # batas ulasan paling tinggi

    # Iterasi
    while awal <= akhir:
        # Pemformatan kueri
        fquery = f'[{{"operationName":"productReviewList","variables":{{"productID":"{idpr}","page":{awal},"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n      }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n      }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n      }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n      }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n      }}\\n      badRatingReasonFmt\\n      __typename\\n    }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n    }}\\n    hasNext\\n    totalReviews\\n    __typename\\n  }}\\n}}\\n"}}]'

        # Make request
        response = requests.post(turldoma, headers=header, data=fquery)

        # Check for HTTP errors
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            break

        try:
            daftar_ulasan = response.json()[0]['data']['productrevGetProductReviewList']['list']
            if not daftar_ulasan:
                break  # Exit loop if no reviews are returned

            for review in daftar_ulasan:
                daftar_teks.append(review['message'])
                daftar_nilai.append(review['productRating'])
                if review['productRating'] == 0:
                    daftar_label.append('total disappointment')
                elif review['productRating'] == 1:
                    daftar_label.append('so inferior')
                elif review['productRating'] == 2:
                    daftar_label.append('very low')
                elif review['productRating'] == 3:
                    daftar_label.append('average')
                elif review['productRating'] == 4:
                    daftar_label.append('good')
                elif review['productRating'] == 5:
                    daftar_label.append('excellent')
                else:
                    daftar_label.append('No score')
            awal += 1

        except (KeyError, IndexError) as e:
            print(f"Error parsing JSON response: {e}")
            break

    return daftar_teks, daftar_nilai, daftar_label
#ke eksekusi

In [99]:
#Proses pembentukan daftar dari enam produk untuk disatukan (Bagian 1)
teks1, nilai1, label1 = datagenerator(urla, ida) #ProdukA
teks2, nilai2, label2 = datagenerator(urlb, idb) #ProdukB
teks3, nilai3, label3 = datagenerator(urlc, idc) #ProdukC
#ke bagian 2

In [100]:
#Bagian 2
teks4, nilai4, label4 = datagenerator(urld, idd) #ProdukD
#ke bagian 3

In [101]:
#Bagian 3
teks5, nilai5, label5 = datagenerator(urle, ide) #ProdukE
teks6, nilai6, label6 = datagenerator(urlf,idf)

### Penyatuan ke satu data

In [102]:
#inspeksi bentuk data
print(teks1) #teks pesan
print(nilai1) #nilai
print(label1) #label

['barang bagus kualitas nya top harga nya naik overall 100 dijamin ori', 'modelnya keren, bahannya bagus cuma agak kepanjangan. perlu dipermak dikit.', 'tinggi 175 bb 60 ukuran L cocok si, pas dan enak banget bahannya adem, murah lagi', 'enak di pake adem dan kece pas di badan gw. tapi kayaknya mudah lecek. Nyaman dipakai Modelnya keren 😎 Jahitannya rapi Ukuran sesuai Warnanya mantap Bahannya adem Kancing tidak mudah lepas Produk sesuai dengan foto Kualitas produk bagus', 'APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa ragu kualitas Erigo ganteng banget sumpah, kemeja nya ga kaku tapi juga ga kerut bahan adem lembut halus gmn yah Kyk ga mau Di lepas kalau pakai ini kemeja. Sukses terus erigo pertahankan ya kualitas barang udh ganteng banget ini mah harga setara 200 rb. Ydh itu aja review nya', 'oke paket di terima dengan baik ,, untuk kualitas barang oke kok kak 😁 bagus ,, tapi sayang nya untuk pengiriman agak lama kak oke itu saja dari saya', 'pas, bahannya enak banget dibadan dan juga 

In [103]:
#deklarasi total ulasan
sampteks = teks1 + teks2 + teks3 + teks4 + teks5 + teks6 #bagian teks
sampnilai = nilai1 + nilai2 + nilai3 + nilai4 + nilai5 + nilai6 #bagian nilai
samplabel = label1 + label2 + label3 + label4 + label5 + label6 #bagian label
#ke penyatuan

enam ulasan ini berasal dari enam produk berasal dari Erigo

In [104]:
#penyatuan ke dataframe
dfori = pd.DataFrame({'Pesan': sampteks,'Nilai': sampnilai,'Label': samplabel})
dfkopi = dfori.copy()
#cek panjang
lendfkop = len(dfkopi)
print(lendfkop)

2210


#### Inspeksi hasil data dari kesalahan

In [105]:
#cek sampel data
kepdfkopi = dfkopi.head(7)
kepdfkopi

,Pesan,Nilai,Label
0,barang bagus kualitas nya top harga nya naik o...,5,excellent
1,"modelnya keren, bahannya bagus cuma agak kepan...",5,excellent
2,"tinggi 175 bb 60 ukuran L cocok si, pas dan en...",5,excellent
3,enak di pake adem dan kece pas di badan gw. ta...,5,excellent
4,APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa r...,5,excellent
5,"oke paket di terima dengan baik ,, untuk kuali...",5,excellent
6,"pas, bahannya enak banget dibadan dan juga ker...",5,excellent


In [106]:
#cek kolom
dfkopinfo = dfkopi.info()
print(dfkopinfo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Pesan   2210 non-null   object
 1   Nilai   2210 non-null   int64 
 2   Label   2210 non-null   object
dtypes: int64(1), object(2)
memory usage: 51.9+ KB
None


#### Pembersihan data (bebas dari data duplikat dan kosong)

In [107]:
#Data Kosong
totkos = dfkopi.isnull().sum() #Kosong
totkos

,0
Pesan,0
Nilai,0
Label,0


In [108]:
#inspeksi data hilang
tothil = dfkopi.isna().sum() #Hilang
print(tothil)

Pesan    0
Nilai    0
Label    0
dtype: int64


In [109]:
#inspeksi data duplikat
dfkopi.duplicated().sum()

461

In [122]:
#Pembersihan data duplikat
dfkopi.drop_duplicates(inplace=True)
dfkopitung = dfkopi.copy()
kep2 = dfkopitung.head(7) #pasca pembersihan
print(kep2)

                                               Pesan  Nilai      Label
0  barang bagus kualitas nya top harga nya naik o...      5  excellent
1  modelnya keren, bahannya bagus cuma agak kepan...      5  excellent
2  tinggi 175 bb 60 ukuran L cocok si, pas dan en...      5  excellent
3  enak di pake adem dan kece pas di badan gw. ta...      5  excellent
4  APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa r...      5  excellent
5  oke paket di terima dengan baik ,, untuk kuali...      5  excellent
6  pas, bahannya enak banget dibadan dan juga ker...      5  excellent


In [111]:
#cek panjang
lendfkop2 = len(dfkopitung)
print(lendfkop2)

1749


# Persiapan tahapan ulasan

## Penataan ulang kata-kata

In [113]:
#pembuatan daftar kata-kata bahasa gaul
kata_gaul = ['yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'untuk', 'dengan', 'saya', 'ada', 'akan', 'pada', 'seperti', 'karena', 'atau', 'jika', 'tidak', 'dalam','iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"] #daftar kata gaul
#ke Bahasa gaul yang populer

In [75]:
#Deklarasi kamus gaul
kamus_gaul =  {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "indo": "indonesia", "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber", "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin", "ga": "tidak", "gak": "tidak", "udah": "sudah", "mau": "ingin", "malah": "justru", "yg": "yang"}

## Tahapan evaluasi kata-kata ke nilai

### Tahap Awal

In [114]:
#Kumpulan fungsi tahap 1 [Pembersihan dan penyeragaman]
def textcleaner (text): #Tahap 1
   text = re.sub(r'@[A-Za-z0-9]+', '', text) #penghilangan tanda at
   text = re.sub(r'#[A-Za-z0-9]+', '', text) #penghilangan tanda pagar
   text = re.sub(r'RT[\s]', '', text) #Bagian RE
   text = re.sub(r"http\S+", '', text) #Bagian penetralan HTTP
   text = re.sub(r'[0-9]+', '', text)
   text = re.sub(r'[^\w\s]', '', text)

   text = text.replace('\n', ' ')
   text = text.translate(str.maketrans('', '', string.punctuation))
   text = text.strip(' ')
   return text
#konversi ke huruf kecil
def casefolding_text(text):
    text = text.lower()
    return text

def tokenizing_text(text):
    text = word_tokenize(text)
    return text

#penyaringan
def filtering_text(text):
    listStopwordsid = set(stopwords.words('indonesian'))
    listStopwordsen = set(stopwords.words('english'))
    listStopwordsid.update(listStopwordsen)
    listStopwordsid.update(kata_gaul)
    filtrat = []
    for txt in text:
        if txt not in listStopwordsid:
            filtrat.append(txt)
    text = filtrat
    return text
def stemming_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()


    words = text.split()


    stemmed_words = [stemmer.stem(word) for word in words]


    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def to_sentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [115]:
#manajemen kata-kata gaul
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in kamus_gaul:
            fixed_words.append(kamus_gaul[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

### Persiapan filtrasi dari kolom pesan

In [125]:
#pengambilan kolom pesan
isipes = dfkopitung['Pesan'].copy() #cek pesan
nomor = dfkopitung['Nilai'].copy() #cek nilai
kategori = dfkopitung['Label'].copy() #cek Label